In [196]:
import pandas as pd
import numpy as np

import sys
import glob
from dataclasses import make_dataclass
import re

Препроцессинг для генерации Игоря

In [197]:
Text = make_dataclass("Text", [("title", str), ("text", str), ("label", int), ("url", str)])

In [198]:
def clean(text):
    def include_del_words(line: str):
        for word in del_words:
            if word in line:
                return True
        return False
    
    del_words = ["текст", "реклама"]
    lines = text.split("\n")
    lines = [line for line in lines if not include_del_words(line)]
    return text

In [199]:
def find_title(text: str, file_name: str):
    title = None
    match = re.search(r"\*\*(.*)\*\*", text)
    if match:
        title = match.group(1).strip()

    first_line, second_line = text.split("\n")[:2]
    if second_line == "":
        if title is None:
            title = first_line
        elif title not in first_line:
            title = None
            print(f'Несколько вариантов title. file: {file_name}', file=sys.stderr)
    
    if title and len(title) > 100:
        print(f'Название слишком длинное. Странно. file: {file_name}', file=sys.stderr)
        title = None

    if title is None:
        print(f'Название текста не найдено. file: {file_name}', file=sys.stderr)
    return title

def cut_title(text: str, title: str, file_name: str):
    if title is None:
        return text
    lines = text.split("\n")
    lines = [line for line in lines if line != ""]
    if title in lines[0]:
        return "\n".join(lines[1:])
    else:
        print(f'Проблемы с обрезанием названия. file: {file_name}', file=sys.stderr)
        return text

def preprocess(text: str, file_name):
    text = clean(text).strip("\n\t\#")
    title = find_title(text, file_name)
    text = cut_title(text, title, file_name)
    return Text(title, text, 1, "Игорь")

In [200]:
file_name = "generated_data/text092.txt"
with open(file_name, 'r') as file:
        file_text = file.read()
        preprocess(file_text, file_name)

Название слишком длинное. Странно. file: generated_data/text092.txt
Название текста не найдено. file: generated_data/text092.txt


In [201]:
Text = make_dataclass("Text", [("title", str), ("text", str), ("label", int), ("url", str)])
texts = []

file_pattern = 'generated_data/text*.txt'
file_list = glob.glob(file_pattern)

for file_path in file_list:
    with open(file_path, 'r') as file:
        file_text = file.read()
        try:
            new_text = preprocess(file_text, file_path)
            texts.append(new_text)
        except Exception as e:
            print("Error: ", e)


data = pd.DataFrame(texts)
print(len(data))

102


Название слишком длинное. Странно. file: generated_data/text092.txt
Название текста не найдено. file: generated_data/text092.txt
Название слишком длинное. Странно. file: generated_data/text093.txt
Название текста не найдено. file: generated_data/text093.txt
Название слишком длинное. Странно. file: generated_data/text090.txt
Название текста не найдено. file: generated_data/text090.txt
Название слишком длинное. Странно. file: generated_data/text068.txt
Название текста не найдено. file: generated_data/text068.txt
Название слишком длинное. Странно. file: generated_data/text040.txt
Название текста не найдено. file: generated_data/text040.txt
Название слишком длинное. Странно. file: generated_data/text054.txt
Название текста не найдено. file: generated_data/text054.txt
Название слишком длинное. Странно. file: generated_data/text018.txt
Название текста не найдено. file: generated_data/text018.txt
Название слишком длинное. Странно. file: generated_data/text024.txt
Название текста не найдено. f

In [202]:
data.to_csv("generated_data/prpr_generated_data")